# Исследование надежности заемщиков.


Целью данного исследования является проанализировать статистику платежеспособности клиентов банка, чтобы определить их надежность для целей создания кредитного скоринга. Для этого необходимо решить следующие задачи:
    
1) изучить данные и и специфику;
    
2) осуществить их предварительную обработку для того, чтобы результаты анализа отражали реальные закономерности, а не искаженные вследствие наличия в данных неточностей или ошибок;
    
3) проанализировать подготовленные данные, чтобы ответить на следующие вопросы:
    
-Есть ли зависимость между количеством детей и возвратом кредита в срок?
    
-Есть ли зависимость между семейным положением и возвратом кредита в срок?
    
-Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
    
-Как разные цели кредита влияют на его возврат в срок?</div>

### Шаг 1. Обзор данных

In [256]:
import pandas as pd

читаем файл с данными:

In [257]:
data = pd.read_csv('/datasets/data.csv')

проверяем наличие и количество пропущенных значений:

In [258]:
data.info()
data.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

### Шаг 2.1 Заполнение пропусков

Прошлое действие помогло выявить пропуски в двух столбцах: "days_employed" и "total_income". 
Исходя из смысла данных столбцов, в них должна содержатся информация в формате количественных значений.
Чтобы понять как исправить наличие пропусков нужно определить как их количество может повлиять на результат исследования.
Для этого необходмо посчитать долю пропущенных значений в каждом из столбцов с пропусками:

In [259]:
volume_missed_meanings_de = data["days_employed"].isna().sum() / len(data["days_employed"])
print("Доля пропущенных значений в столбце days_employed: {:.1%}".format(volume_missed_meanings_de))
volume_missed_meanings_ti = data["total_income"].isna().sum() / len(data["total_income"])
print("Доля пропущенных значений в столбце total_income: {:.1%}".format(volume_missed_meanings_ti))

Доля пропущенных значений в столбце days_employed: 10.1%
Доля пропущенных значений в столбце total_income: 10.1%


Получим что доля пропусков в каждом столбце составляет чуть больше 10%, это небольшой процент, но не ничтожный,поэтому им нельзя просто принебречь.
Доля пропущенных значений одинакова в двух стлбцах, возможно это связано с технической ошибкой - нужно уточнить у поставщика данных возможные причины.
Для начала нужно проверить данные на аномалии:

### Шаг 2.2 Проверка данных на аномалии и исправления.

Проверим уникальные значения в числовых типах данных:

In [260]:
data['days_employed'].unique()

array([-8437.67302776, -4024.80375385, -5623.42261023, ...,
       -2113.3468877 , -3112.4817052 , -1984.50758853])

In [261]:
data['total_income'].unique()

array([253875.6394526 , 112080.01410244, 145885.95229686, ...,
        89672.56115303, 244093.05050043,  82047.41889948])

In [262]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [263]:
data['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

Заменим отрицательные числа на положительные в стоблце стаж и количество детей - для этого создадим функцию:

In [264]:
data['days_employed'] = data['days_employed'].abs()
data['children'] = data['children'].abs()
data['days_employed'].unique()

array([8437.67302776, 4024.80375385, 5623.42261023, ..., 2113.3468877 ,
       3112.4817052 , 1984.50758853])

In [265]:
data['children'].unique()

array([ 1,  0,  3,  2,  4, 20,  5])

Учитывая колличественный тип данных в этих столбцах и их контекст, наиболее оптимальным способ исправить 
наличие пропусков в столбцах - это заполнить их медианным значением:

In [266]:
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].median())
data['total_income'] = data['total_income'].fillna(data['total_income'].median())

При проверке уникальных значений столбца children было замечено уникальное значение 20, предположительно это
следствие человеческого фактора (опечатка), заменим его на медианное значение:

In [267]:
children_median = data.loc[data.loc[:, 'children'] != 20]['children'].median()
data['children'] = data['children'].replace(20, children_median)

Убедимся что пропусков не осталось:

In [268]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  float64
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(3), int64(4), object(5)
memory usage: 2.0+ MB


### Шаг 2.3. Изменение типов данных.

Из предыдущего действия мы видим, что некоторые столбцы имеют вещественный тип данных. Изменим его на целочисленный:

In [269]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')

Убедимся что тип данных изменился:

In [270]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  float64
 1   days_employed     21525 non-null  int64  
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int64  
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


### Шаг 2.4. Удаление дубликатов.

Проверим данные на наличие неявных дубликатов в столбцах со строковыми значениями:
- данные об образовании:

In [274]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

в столбце присутствуют данные, записанные разным регистром, исправим это:

In [275]:
data['education'] = data['education'].str.lower()

- данные о семейном статусе:

In [276]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

в столбце присутствуют данные, записанные разным регистром, исправим это:

In [277]:
data['family_status'] = data['family_status'].str.lower()

- данные о гендерной принадлежности:

In [278]:
data['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

обнаружен гендер XNA, проверим его количество:

In [279]:
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

гендер XNA встречается 1 раз в стоблце, имеющем 21525 значений, и не влияет на результаты исследования.
Предположительно в данном случае данные просто отсутствут в связи с человеческим фактором.
- данные о типе дохода:

In [280]:
data['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

данные о типе дохода не содержат неявных дубликатов
- данные о цели займа:

In [281]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Исследуем данные на наличие дубликатов:

In [282]:
data.duplicated().sum()

71

данные содержат 71 явных дубликата. Удалим их, перезаписав индексы:

In [283]:
data = data.drop_duplicates().reset_index(drop=True)

проверим выполнение предыдщего действия:

In [284]:
data.duplicated().sum()

0

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Чтобы облегчить восприятие таблицы сформируем дополнительные датафреймы, соединяющие по смыслы id и значение 
в колонках образование и семейный статус:

In [285]:
id_ed_meaning ={'education':data['education'].unique(), 'education_id':data['education_id'].unique()}
id_fs_meaning ={'family_status':data['family_status'].unique(), 'family_status_id':data['family_status_id'].unique()}
pd.DataFrame(id_ed_meaning)

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [286]:
pd.DataFrame(id_fs_meaning)

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


Удалим из основной таблицы стоблцы family_status и education:

In [287]:
data_simplif = data.drop(columns = ['education', 'family_status'], axis = 1)
data_simplif.head(5)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1.0,8437,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1.0,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0.0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3.0,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0.0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


### Шаг 2.6. Категоризация дохода.

Категоризируем доходы по их величине:

In [288]:
def total_income_categoriser(income_value):
    if income_value >= 0:
        if income_value <= 30000:
            return 'E'
    if income_value >= 30001:
        if income_value <= 50000:
            return 'D'
    if income_value >= 50001:
        if income_value <= 200000:
            return 'C'
    if income_value >= 200001:
        if income_value <= 1000000:
            return 'B'
    return 'A'
data_simplif['total_income_category'] = data_simplif['total_income'].apply(total_income_categoriser)
data_simplif.head(5)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1.0,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1.0,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0.0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3.0,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0.0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


### Шаг 2.7. Категоризация целей кредита.

Категоризируем данные по цели кредита. Исходя из данных шага 2.4 мы видим, что цели кредита содержит множество схожих значений,
совпадающих по смыслу. Их необходимо лемматизировать:

In [289]:
from pymystem3 import Mystem
m = Mystem()

def lemmas(purpose):
    lemma = m.lemmatize(purpose)
    return lemma

создаем новый столбец со значением лемматизации колонки lemm_results.

In [290]:
data_simplif['lemm_results'] = data_simplif['purpose'].apply(lemmas)
data_simplif.head(5)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,lemm_results
0,1.0,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B,"[покупка, , жилье, \n]"
1,1.0,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,"[приобретение, , автомобиль, \n]"
2,0.0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C,"[покупка, , жилье, \n]"
3,3.0,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,"[дополнительный, , образование, \n]"
4,0.0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,"[сыграть, , свадьба, \n]"


На основе лемматизации категоризируем значения цели кредита, создав специальную функцию:

In [291]:
def lemm_results(rows):
    if 'автомобиль' in rows:
        return 'операции с автомобилем'
    if 'свадьба' in rows:
        return 'проведение свадьбы'  
    if 'образование' in rows:
        return 'получение образования'
    if 'жилье' or 'недвижимость' in rows:
        return 'операции с недвижимостью'   
data_simplif['purpose_category'] = data_simplif['lemm_results'].apply(lemm_results)
data_simplif.head(5)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,lemm_results,purpose_category
0,1.0,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B,"[покупка, , жилье, \n]",операции с недвижимостью
1,1.0,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,"[приобретение, , автомобиль, \n]",операции с автомобилем
2,0.0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C,"[покупка, , жилье, \n]",операции с недвижимостью
3,3.0,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,"[дополнительный, , образование, \n]",получение образования
4,0.0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,"[сыграть, , свадьба, \n]",проведение свадьбы


удалим рабочий столбец с результатами лемматизации:

In [292]:
data_final = data_simplif.drop(columns = ['lemm_results'], axis = 1)
data_final.head(5)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1.0,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1.0,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0.0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3.0,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0.0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


### Ответы на вопросы.

##### Вопрос 1:  Есть ли зависимость между количеством детей и возвратом кредита в срок?

Для оценки имеющихся данных будет удобным использовать сводную таблицу:

In [293]:
children_debt = data_final.pivot_table(index = 'children', values = 'debt', aggfunc='sum')
children_debt.sort_values(by = 'debt', ascending = False)

,debt
children,
0.0,1071
1.0,445
2.0,194
3.0,27
4.0,4
5.0,0


In [294]:
children_debt = data_final.pivot_table(index = 'children', values = 'debt', aggfunc=['sum','count','mean'])
children_debt.sort_values(by = [('sum', 'debt')], ascending = False)

,sum,count,mean
,debt,debt,debt
children,,,
0.0,1071,14167,0.075598
1.0,445,4855,0.091658
2.0,194,2052,0.094542
3.0,27,330,0.081818
4.0,4,41,0.097561
5.0,0,9,0.000000


##### Вывод 1: исходя из анализа данных, следует, что задолженность встречается чаще у заемщиков без детей. Однако если выделить долю наличия задолженности ко всему числу задолженности группе, мы увидим, что наиболее часто трудности с возвратом кредита в срок возникают у заемщиков с 4 детьми. В то же время наличие задолженности прямо не зависит от количества детей, поскольку мы видим, что у заемщиков с 3 детьми задолженность встречается реже, чем у заемщиков с 2 детьми.

##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

Для оценки имеющихся данных будет удобным использовать сводную таблицу:

In [295]:
family_debt = data_final.pivot_table(index = 'family_status_id', values = 'debt', aggfunc='sum')
family_debt_sorted = family_debt.sort_values(by = 'debt', ascending = False)
family_debt_sorted

,debt
family_status_id,
0,931
1,388
4,274
3,85
2,63


In [296]:
family_debt = data_final.pivot_table(index = 'family_status_id', values = 'debt', aggfunc=['sum','count','mean'])
family_debt.sort_values(by = [('sum', 'debt')], ascending = False)

,sum,count,mean
,debt,debt,debt
family_status_id,,,
0,931,12339,0.075452
1,388,4151,0.093471
4,274,2810,0.097509
3,85,1195,0.071130
2,63,959,0.065693


In [297]:
pd.DataFrame(id_fs_meaning)

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


##### Вывод 2: исходя из анализа данных, вероятность наличия задолженности чаще встречается у заемщиков, имеющих семейный статус "не женат/не замужем", реже всего у  заемщиков, имеющих семейный статус "вдовец/вдова".

##### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок? 

Для оценки имеющихся данных будет удобным использовать сводную таблицу:

In [298]:
total_income_category_debt = data_final.pivot_table(index = 'total_income_category', values = 'debt', aggfunc='sum')
total_income_category_debt_sorted = total_income_category_debt.sort_values(by = 'debt', ascending = False)
total_income_category_debt_sorted

,debt
total_income_category,
C,1360
B,356
D,21
A,2
E,2


In [299]:
total_income_category_debt = data_final.pivot_table(index = 'total_income_category', values = 'debt', aggfunc=['sum','count','mean'])
total_income_category_debt_sorted = total_income_category_debt.sort_values(by = [('sum', 'debt')], ascending = False)
total_income_category_debt_sorted

,sum,count,mean
,debt,debt,debt
total_income_category,,,
C,1360,16016,0.084915
B,356,5041,0.070621
D,21,350,0.060000
A,2,25,0.080000
E,2,22,0.090909


##### Вывод 3: исходя из анализа данных, вероятность наличия задолженности чаще встречается у заемщиков, имеющих доходы категории "E" и "C", то есть доходы 0–30000 и 50001–200000 условных едениц, соответственно.

##### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

Для оценки имеющихся данных будет удобным использовать сводную таблицу:

In [300]:
purpose_category_debt = data_final.pivot_table(index = 'purpose_category', values = 'debt', aggfunc='sum')
purpose_category_debt.sort_values(by = 'debt', ascending = False)

,debt
purpose_category,
операции с недвижимостью,782
операции с автомобилем,403
получение образования,370
проведение свадьбы,186


In [301]:
purpose_category_debt = data_final.pivot_table(index = 'purpose_category', values = 'debt', aggfunc=['sum','count','mean'])
purpose_category_debt.sort_values(by = [('sum', 'debt')], ascending = False)

,sum,count,mean
,debt,debt,debt
purpose_category,,,
операции с недвижимостью,782,10811,0.072334
операции с автомобилем,403,4306,0.093590
получение образования,370,4013,0.092200
проведение свадьбы,186,2324,0.080034


##### Вывод 4: исходя из анализа данных, вероятность наличия задолженности чаще встречается у заемщиков, берущих кредит на операции с автомобилем, реже у заемщиков, берущих кредит на операции с недвижимостью.

## Общий вывод:

#### Для подготовки анализа,  направленного на определение надежности заемщиков с точки зрения их платежеспособности, были предприняты следующие шаги:
* 1. изучены данные о платежеспособности клиентов банка и их специфика, в результате чего были выявлены пропуски в двух столбцах: "days_employed" и "total_income";

* 2. осуществили предварительную обработку данных для того, чтобы результаты анализа отражали реальные закономерности, а не искаженные вследствие наличия в данных неточностей или ошибок, в частности: были выявлены и устранены аномалии, связанные со значениями некоторых столбцов, заменены обнаруженные ранее пропуски, выявлены и удалены ненявные и явные дубликаты, осуществлена лемматизация и категоризация данных;

* 3. перечисленные действия позволили нам произвести корректное построение сводных таблиц для определение доли искомых значений

#### В результате чего удалось выявить следующие особенности:

* Вероятность погашения кредита в срок напрямую не коррелирует с количеством детей, то есть не изменяется непосредственно в связи с изменением количества детей. Однако наиболее часто трудности с возвратом кредита в срок возникают у заемщиков с 4 детьми.
* Вероятность наличия задолженности чаще встречается у заемщиков, имеющих семейный статус "не женат/не замужем".
* Вероятность наличия задолженности чаще встречается у заемщиков, имеющих доходы категории "E" и "C", то есть доходы 0–30000 и 50001–200000 условных едениц, соответственно.
* Вероятность наличия задолженности чаще встречается у заемщиков, берущих кредит на операции с автомобилем, реже у заемщиков, берущих кредит на операции с недвижимостью. Потенциально это может быть обусловлено наличием залога цели кредита в пользу банка, что непосредствено побуждает заемщиков, берущих кредит на операции с недвижимостью, к своевременному возрату кредита, поскольку для них риск потерять недвижимость будет болезненей нежели риск потерять автомобиль.